In [1]:
import pandas as pd
import numpy as np
from  sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv("C:\\Users\\266251\\Machine Learning\\Upvote-Vidhya\\train_NIR5Yl1.csv")

In [3]:
test=pd.read_csv("C:\\Users\\266251\\Machine Learning\\Upvote-Vidhya\\test_8i3B3FC.csv")

In [4]:
labelencoder_X = LabelEncoder()
train['Tag'] = labelencoder_X.fit_transform(train['Tag'])
train.drop(['ID','Username'], axis=1,inplace =True)
target = train['Upvotes']


In [5]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([train['Answers']])[0]
train['pd_watched'] = pd_watched

In [6]:
feature_names = [x for x in train.columns if x not in ['Upvotes']]

In [7]:
x_train, x_val, y_train, y_val = train_test_split(train[feature_names], target,test_size = 0.22,random_state =205)

In [8]:
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_val = sc_X.transform(x_val)

In [9]:
poly_reg = PolynomialFeatures(degree = 4,interaction_only=False, include_bias=True)
X_poly = poly_reg.fit_transform(x_train)
poly_reg.fit(x_train, y_train)
lin_reg_1 = linear_model.LassoLars(alpha=0.021,max_iter=150)
lin_reg_1.fit(X_poly, y_train)

LassoLars(alpha=0.021, copy_X=True, eps=2.220446049250313e-16,
     fit_intercept=True, fit_path=True, max_iter=150, normalize=True,
     positive=False, precompute='auto', verbose=False)

In [10]:
pred_val = lin_reg_1.predict(poly_reg.fit_transform(x_val))

In [11]:
print(r2_score(y_val, pred_val))

0.910533035685813


In [12]:
ids = test['ID']
test.drop(['ID','Username'], axis=1,inplace =True)

In [13]:
labelencoder_X = LabelEncoder()
test['Tag'] = labelencoder_X.fit_transform(test['Tag'])

In [14]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([test['Answers']])[0]
test['pd_watched'] = pd_watched

In [15]:
test = sc_X.fit_transform(test)

In [16]:
pred_test = lin_reg_1.predict(poly_reg.fit_transform(test))

In [17]:
pred_test=abs(pred_test)

In [18]:
submission = pd.DataFrame({'ID': ids,
                           'Upvotes':pred_test
                           })

In [19]:
submission.to_csv("C:\\Users\\266251\\Machine Learning\\Upvote-Vidhya\\final_sub477.csv",index=False)